# MAFFT Example

Before running we need to install MAFFT and a wrapper for it in Python. First, install MAFFT:
> [MAFFT installation instructions](https://mafft.cbrc.jp/alignment/software/)

Then you want to install BioPython (which has a wrapper for MAFFT). If using Anaconda run:
> conda install biopython

In [1]:
from Bio.Align.Applications import MafftCommandline
from io import StringIO
from Bio import AlignIO
import numpy as np

In [2]:
def MAFFT(in_file):
    mafft_cline = MafftCommandline(input=in_file,localpair=True)
    # the above requires using command line
    # output to a python datatype
    stdout, stderr = mafft_cline()
    return AlignIO.read(StringIO(stdout), "fasta")

In [3]:
# Create alignment
in_file = "ex.fasta"
align = MAFFT(in_file)

# print resulting alignment
for element in align:
    print(element.seq)
# alt print(align)

---act
gata--
--tag-
--tac-


# Alignment Scoring

Compute sum-of-pairs-like scores. First, scoring based on BLOSUM for protein sequences (the following code is from Petti et al.: [make_pairwise_aln_figures.ipynb](https://github.com/spetti/SMURF/blob/main/examples/LAM_AF_examples/make_pairwise_aln_figures.ipynb))

In [4]:
# BLOSUM62 from https://www.ncbi.nlm.nih.gov/IEB/ToolBox/C_DOC/lxr/source/data/BLOSUM62   
array_as_str = "4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0 -2 -1 -1 -1 -4"\
" -1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3 -1 -2  0 -1 -4"\
" -2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3  4 -3  0 -1 -4"\
" -2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3  4 -3  1 -1 -4"\
" 0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -3 -1 -3 -1 -4"\
" -1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2  0 -2  4 -1 -4"\
" -1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2  1 -3  4 -1 -4"\
" 0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3 -1 -4 -2 -1 -4"\
" -2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3  0 -3  0 -1 -4"\
" -1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3 -3  3 -3 -1 -4"\
" -1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1 -4  3 -3 -1 -4"\
" -1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2  0 -3  1 -1 -4"\
" -1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1 -3  2 -1 -1 -4"\
" -2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1 -3  0 -3 -1 -4"\
" -1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2 -2 -3 -1 -1 -4"\
" 1 -1  1  0 -1  0  0  0 -1 -2 -2  0 -1 -2 -1  4  1 -3 -2 -2  0 -2  0 -1 -4"\
" 0 -1  0 -1 -1 -1 -1 -2 -2 -1 -1 -1 -1 -2 -1  1  5 -2 -2  0 -1 -1 -1 -1 -4"\
" -3 -3 -4 -4 -2 -2 -3 -2 -2 -3 -2 -3 -1  1 -4 -3 -2 11  2 -3 -4 -2 -2 -1 -4"\
" -2 -2 -2 -3 -2 -1 -2 -3  2 -1 -1 -2 -1  3 -3 -2 -2  2  7 -1 -3 -1 -2 -1 -4"\
" 0 -3 -3 -3 -1 -2 -2 -3 -3  3  1 -2  1 -1 -2 -2  0 -3 -1  4 -3  2 -2 -1 -4"\
" -2 -1  4  4 -3  0  1 -1  0 -3 -4  0 -3 -3 -2  0 -1 -4 -3 -3  4 -3  0 -1 -4"\
" -1 -2 -3 -3 -1 -2 -3 -4 -3  3  3 -3  2  0 -3 -2 -1 -2 -1  2 -3  3 -3 -1 -4"\
" -1  0  0  1 -3  4  4 -2  0 -3 -3  1 -1 -3 -1  0 -1 -2 -2 -2  0 -3  4 -1 -4"\
" -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -4"\
" -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  1"\

# make this into a matrix
BLOSUM_array=np.array([int(_) for _ in array_as_str.split(" ") if _ !='']).reshape((25,25))

# as a dictionary to make it easy w/ inputs
aa_s=[_ for _ in "A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  B  J  Z  X  *".split(" ") if _ !='']
BLOSUM_d={}
for n1, aa1 in enumerate(aa_s):
    for n2, aa2 in enumerate(aa_s):
         BLOSUM_d[(aa1, aa2)]=BLOSUM_array[n1,n2]
            
# unit gap penalty
for aa in aa_s:
    BLOSUM_d[(aa,"-")] = -1
    BLOSUM_d[("-",aa)] = -1
BLOSUM_d[("-","-")] = 0

def count_gap_prefix(seq):
    gap_count = 0
    for ch in seq:
        if type(ch) == int and ch <0: gap_count +=1
        elif ch == '-': gap_count +=1
        else: return gap_count
    return gap_count

# get pairwise
def get_blosum_score(seq1, seq2):
    start = min(count_gap_prefix(seq1), count_gap_prefix(seq2))
    end = min(count_gap_prefix(seq1[::-1]), count_gap_prefix(seq2[::-1]))
    score = 0
    for n in range(start, len(seq1)- end):
        score += BLOSUM_d[(seq1[n], seq2[n])]
    return score

# if aligned appropriately +1 else -1
def get_unit_score(seq1, seq2):
    start = min(count_gap_prefix(seq1), count_gap_prefix(seq2))
    end = min(count_gap_prefix(seq1[::-1]), count_gap_prefix(seq2[::-1]))
    score = 0
    for n in range(start, len(seq1)- end):
        if seq1[n] == seq2[n]:
            score += 1
        else:
            score -= 1
    return score

In [5]:
# get each pairwise BLOSUM score for each value in example FASTA file
# while the ex uses DNA the letters are accounted for in the BLOSUM mat
seqs = len(align)
for i in range(seqs):
    for j in range(i+1,seqs):
        seq1 = align[i].seq.upper()
        seq2 = align[j].seq.upper()
        blosum = get_blosum_score(seq1, seq2)
        unit = get_unit_score(seq1, seq2)
        print("BLOSUM score of " + str(seq1.strip('-')) + ' to ' + str(seq2.strip('-')) + ': ' + str(blosum))
        print("Unit score of " + str(seq1.strip('-')) + ' to ' + str(seq2.strip('-')) + ': ' + str(unit))

BLOSUM score of ACT to GATA: -1
Unit score of ACT to GATA: -4
BLOSUM score of ACT to TAG: -1
Unit score of ACT to TAG: -2
BLOSUM score of ACT to TAC: 11
Unit score of ACT to TAC: 0
BLOSUM score of GATA to TAG: 6
Unit score of GATA to TAG: -1
BLOSUM score of GATA to TAC: 6
Unit score of GATA to TAC: -1
BLOSUM score of TAG to TAC: 6
Unit score of TAG to TAC: 1
